## Investigating brain structural maturation in children and adolescents born very preterm using the brain age framework
**Claire Kelly***<sup>1,2,3</sup>, Gareth Ball<sup>3,4</sup>, Lillian G Matthews<sup>1,2,5</sup>, Jeanie LY Cheong<sup>2,6,7</sup>, Lex W Doyle<sup>2,4,6,7</sup>, Terrie E Inder<sup>5</sup>, Deanne K Thompson<sup>2,3,4</sup> & Peter J Anderson<sup>1,2</sup>  

1. Turner Institute for Brain and Mental Health, School of Psychological Sciences, Monash University, Melbourne, Australia
2. Victorian Infant Brain Studies (VIBeS), Murdoch Children’s Research Institute, Melbourne, Australia
3. Developmental Imaging, Murdoch Children’s Research Institute, Melbourne, Australia
4. Department of Paediatrics, The University of Melbourne, Melbourne, Australia
5. Department of Pediatric Newborn Medicine, Brigham and Women’s Hospital, Harvard Medical School, Boston, MA, USA
6. The Royal Women’s Hospital, Melbourne, Australia
7. Department of Obstetrics and Gynaecology, The University of Melbourne, Melbourne, Australia

* claire.kelly@monash.edu 



In [ ]:
# This code was run using python 3.8.2
# Required packages include numpy (version 1.20.1) and sklearn (version 0.24.1)

In [ ]:
import numpy as np

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, DotProduct, WhiteKernel
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

from sklearn.metrics import mean_absolute_error

Participants are from the Victorian Infant Brain Study (VIBeS). Infants born very preterm (<30 weeks’ gestation or <1250 g) were recruited between July 2001 and December 2003 from the Royal Women’s Hospital, Melbourne. The participants were invited to return for MRI at ages 7 and 13 years.  

An independent dataset was used to train the age prediction model. Data were acquired from The Pediatric Imaging, Neurocognition, and Genetics (PING) data repository (Jernigan et al., 2016), made available via the [NIMH Data Archive](https://nda.nih.gov/edit_collection.html?id=2607).  

All T1-weighted images for the PING and VIBeS datasets were preprocessed using the recon-all pipeline from FreeSurfer version 6. Volume, area and thickness of the cortical regions and volume of the subcortical structures were extracted for use as features in the brain age prediction model (see Table 1). 

### Inputs

In [2]:
# Expected model inputs:

# Training data
# ping_freesurfer: n (observations) x p (freesurfer variables) array
# ping_age: n (observations) x p (age variable) array
# ping_site: n (observations) x p (MRI scan site variable) array

# Test data
# vibes7_freesurfer: n (observations) x p (freesurfer variables) array
# vibes7_age: n (observations) x p (age variable) array

# vibes13_freesurfer: n (observations) x p (freesurfer variables) array
# vibes13_age: n (observations) x p (age variable) array

### Brain age model

In [ ]:
# Set up Gaussian Process Regression (GPR) model
ss = StandardScaler()
kernel =  1 * DotProduct(sigma_0=1e-5, sigma_0_bounds='fixed') + RBF(length_scale=[5], length_scale_bounds=(.1,1000)) + WhiteKernel(noise_level=1)
gpr = GaussianProcessRegressor(kernel=kernel, normalize_y=True, alpha=0.0)

# As a pipeline
nonlinear_model = Pipeline(steps=(['scale', ss], ['model', gpr])) 

### Cross-validation

In [ ]:
# Cross validate the GPR model in the PING dataset  

# array for predictions
pred_y = np.zeros(np.shape(ping_age))

# number of folds
folds = 10 

# stratified k-fold cross-validator
skf = StratifiedKFold(n_splits=folds, shuffle=True)

# arrays for R2 and MAE in each folds
fold_r2 = np.zeros((folds,1))
fold_mae = np.zeros((folds,1))

# CV loop
for f,idx in enumerate(skf.split(ping_freesurfer, ping_site)): # stratify by site
    train_index = idx[0]
    test_index = idx[1]
    
    # select train/test data
    X_train, X_test = ping_freesurfer[train_index], ping_freesurfer[test_index]
    y_train, y_test = ping_age[train_index], ping_age[test_index]
    
    # fit model
    nonlinear_model.fit(X_train, y_train)
    # predict test data
    pred_y[test_index] = nonlinear_model.predict(X_test)
   
    # calculate metrics
    fold_r2[f] = nonlinear_model.score(X_test, y_test)
    print("test r2: %.2f" % nonlinear_model.score(X_test, y_test))
    print("train r2: %.2f" % nonlinear_model.score(X_train, y_train))
    #print(gpr.kernel_) 
     
    fold_mae[f] = mean_absolute_error(y_test, pred_y[test_index])
    print("test mae: %.2f" % mean_absolute_error(y_test, pred_y[test_index]))
    

### Cross-validation results

In [ ]:
# Calculate the correlation coefficient between true and predicted age across all PING subjects
cc = np.corrcoef(ping_age, pred_y)[0,1]
print('correlation between age and predicted age: {:02f}'.format(cc))

# Calculate the mean R2 and MAE over the 10 CV folds for the PING dataset 
mean_r2 = np.mean(fold_r2)
sd_r2 = np.std(fold_r2)
print('mean R-squared +/- S.D. over folds: {:03f} +/- {:03f}'.format(mean_r2, sd_r2))

mean_mae = np.mean(fold_mae)
sd_mae = np.std(fold_mae)
print('mean MAE +/- S.D. over folds: {:03f} +/- {:03f}'.format(mean_mae, sd_mae))


### Predicting brain age in VIBeS

In [ ]:
# Refit the GPR model to the full PING dataset, then apply the model to the VIBeS-7 dataset
X_train = ping_freesurfer
y_train = ping_age

X_test = vibes7_freesurfer
y_test = vibes7_age

nonlinear_model.fit(X_train, y_train)
pred_y = nonlinear_model.predict(X_test)

print("r2: %.2f" % nonlinear_model.score(X_test, y_test))
print("mae: %.2f" % mean_absolute_error(y_test, pred_y))
print("coef: %.2f" % np.corrcoef(y_test, pred_y)[0,1])

vibes7_brainage = pred_y - vibes7_age

# Export vibes7_brainage for further statistical analysis

In [ ]:
# Apply the model to the VIBeS-13 dataset
X_test = vibes13_freesurfer
y_test = vibes13_age

pred_y = nonlinear_model.predict(X_test)

print("r2: %.2f" % nonlinear_model.score(X_test, y_test))
print("mae: %.2f" % mean_absolute_error(y_test, pred_y))
print("coef: %.2f" % np.corrcoef(y_test, pred_y)[0,1])

vibes13_brainage = pred_y - vibes13_age

# Export vibes13_brainage for further statistical analysis